In [ ]:
!pip install -U sentence-transformers

In [2]:
import numpy as np
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer, util
import torch, gc

# gc.collect()
# torch.cuda.empty_cache()
# device = torch.device("cpu")

In [ ]:
# print(device)

In [3]:
wiki_data = []
with open('tokenized.txt','r',encoding='utf8') as file:
    for line in enumerate(file):
        wiki_data.append(line)

print(len(wiki_data))

5315384


In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
# model = SentenceTransformer('sentence-transformers/paraphrase-albert-small-v2')

In [5]:
# encode corpus to get corpus embeddings

# gc.collect()
# torch.cuda.empty_cache()

data_embeddings = model.encode(wiki_data, show_progress_bar = True)
# data_embeddings = model.encode(wiki_data,batch_size=1,convert_to_tensor=True,show_progress_bar = True)



In [6]:
#Store sentences & embeddings on disc
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'wiki_data': wiki_data, 'data_embeddings': data_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    wiki_data = stored_data['wiki_data']
    data_embeddings = stored_data['data_embeddings']

print(len(wiki_data))

5315384


In [ ]:
# #Load corpus and embeddings from disc
# with open('embeddings.pkl','rb') as fIn:
#     stored_data = pickle.load(fIn)
#     wiki_data = stored_data['wiki_data']
#     data_embeddings = stored_data['data_embeddings']

In [7]:
input_query = "Motorcycle ||| This is a sound made by the catA motorcycle, often called a motorbike, bike, or cycle, is a two- or three-wheeled motor vehicle. Motorcycle design varies greatly to suit a range of different purposes: long-distance travel, commuting, cruising, sport (including racing), and off-road riding. Motorcycling is riding a motorcycle and being involved in other related social activity such as joining a motorcycle club and attending motorcycle rallies. The 1885 Daimler Reitwagen made by Gottlieb Daimler and Wilhelm Maybach in Germany was the first internal combustion, petroleum-fueled motorcycle. In 1894, Hildebrand & Wolfmüller became the first series production motorcycle. In 2014, the three top motorcycle producers globally by volume were Honda (28%), Yamaha (17%) (both from Japan), and Hero MotoCorp (India).[6] In developing countries, motorcycles are considered utilitarian due to lower prices and greater fuel economy. Of all the motorcycles in the world, 58% are in the Asia-Pacific and Southern and Eastern Asia regions, excluding car-centric Japan."

# encode sentence to get sentence embeddings
sentence_embedding = model.encode(input_query,show_progress_bar=True)


In [8]:

# top_k results to return
top_k=3

# compute similarity scores of the sentence with the corpus
cos_scores = util.pytorch_cos_sim(sentence_embedding, data_embeddings)[0]


In [10]:

# Sort the results in decreasing order and get the first top_k
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("Input Query:", input_query, "\n")
print("Top", top_k, "most similar sentences in corpus:")
for i in top_results[0:top_k]:
    print(wiki_data[i], "(Score: %.4f)" % (cos_scores[i]))
    print('\n')

print("DONE")

Input Query: Motorcycle ||| This is a sound made by the catA motorcycle, often called a motorbike, bike, or cycle, is a two- or three-wheeled motor vehicle. Motorcycle design varies greatly to suit a range of different purposes: long-distance travel, commuting, cruising, sport (including racing), and off-road riding. Motorcycling is riding a motorcycle and being involved in other related social activity such as joining a motorcycle club and attending motorcycle rallies. The 1885 Daimler Reitwagen made by Gottlieb Daimler and Wilhelm Maybach in Germany was the first internal combustion, petroleum-fueled motorcycle. In 1894, Hildebrand & Wolfmüller became the first series production motorcycle. In 2014, the three top motorcycle producers globally by volume were Honda (28%), Yamaha (17%) (both from Japan), and Hero MotoCorp (India).[6] In developing countries, motorcycles are considered utilitarian due to lower prices and greater fuel economy. Of all the motorcycles in the world, 58% are 

In [16]:
input_query_1 = "Programming language ||| A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declarative form (i.e. the desired result is specified, not how to achieve it).The description of a programming language is usually split into the two components of syntax (form) and semantics (meaning). Some languages are defined by a specification document (for example, the C programming language is specified by an ISO Standard) while other languages (such as Perl) have a dominant implementation that is treated as a reference. Some languages have both, with the basic language defined by a standard and extensions taken from the dominant implementation being common"

# encode sentence to get sentence embeddings
sentence_embedding_1 = model.encode(input_query_1,show_progress_bar=True)

# top_k results to return
top_k=3

# compute similarity scores of the sentence with the corpus
cos_scores = util.pytorch_cos_sim(sentence_embedding_1, data_embeddings)[0]


# Sort the results in decreasing order and get the first top_k
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("Input Query:", input_query_1, "\n")
print("Top", top_k, "most similar sentences in corpus:")
for i in top_results[0:top_k]:
    print(wiki_data[i], "(Score: %.4f)" % (cos_scores[i]))
    print('\n')


Input Query: Programming language ||| A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declarati

In [17]:
def InputQuery(title,abstract,top_k):
    input_query = title+" ||| "+abstract
    input_embeddings = model.encode(input_query)
    cos_scores = util.pytorch_cos_sim(input_embeddings, data_embeddings)[0]
    # Sort the results in decreasing order and get the first top_k
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("Input Query:", input_query, "\n")
    print("Top", top_k, "most similar topics in the corpus:")
    for i in top_results[0:top_k]:
        print(wiki_data[i], "(Score: %.4f)" % (cos_scores[i]))
        print('\n')

In [18]:
InputQuery("Programming language ","A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declarative form (i.e. the desired result is specified, not how to achieve it).The description of a programming language is usually split into the two components of syntax (form) and semantics (meaning). Some languages are defined by a specification document (for example, the C programming language is specified by an ISO Standard) while other languages (such as Perl) have a dominant implementation that is treated as a reference. Some languages have both, with the basic language defined by a standard and extensions taken from the dominant implementation being common.")

Motorcycle ||| This is a sound made by the catA motorcycle, often called a motorbike, bike, or cycle, is a two- or three-wheeled motor vehicle. Motorcycle design varies greatly to suit a range of different purposes: long-distance travel, commuting, cruising, sport (including racing), and off-road riding. Motorcycling is riding a motorcycle and being involved in other related social activity such as joining a motorcycle club and attending motorcycle rallies. The 1885 Daimler Reitwagen made by Gottlieb Daimler and Wilhelm Maybach in Germany was the first internal combustion, petroleum-fueled motorcycle. In 1894, Hildebrand & Wolfmüller became the first series production motorcycle. In 2014, the three top motorcycle producers globally by volume were Honda (28%), Yamaha (17%) (both from Japan), and Hero MotoCorp (India).[6] In developing countries, motorcycles are considered utilitarian due to lower prices and greater fuel economy. Of all the motorcycles in the world, 58% are in the Asia-P